In [1]:
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri

import sagemaker

sagemaker_session = sagemaker.Session()
bucket=sagemaker_session.default_bucket()
region = sagemaker_session.boto_session.region_name
role = sagemaker.get_execution_role()

In [2]:
hyperparameters = {'num_train_epochs': 5, 'save_steps':1000, 'train_batch_size':32, 'eval_batch_size':8}


In [3]:
#git_config = {'repo': 'https://github.com/awslabs/amazon-sagemaker-examples.git', 'branch': 'training-scripts'}

In [4]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                    role=role,
                    framework_version='1.1.0',
                    train_instance_count=1,
                    train_instance_type='ml.p3.8xlarge',
                    source_dir='email-compliance-bert',
                    #git_config=git_config,
                    hyperparameters=hyperparameters)

In [ ]:
estimator.fit('s3://sagemaker-us-east-1-665028136136/compliance-data/batch2/')

2019-09-20 11:57:25 Starting - Starting the training job...
2019-09-20 11:57:27 Starting - Launching requested ML instances......
2019-09-20 11:58:27 Starting - Preparing the instances for training......
2019-09-20 11:59:34 Downloading - Downloading input data
2019-09-20 11:59:34 Training - Downloading the training image...
2019-09-20 12:00:15 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-09-20 12:00:15,962 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-09-20 12:00:16,006 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-09-20 12:00:17,416 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-09-20 12:00:17,662 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-09-20 12:00:17,663 sagemaker-c

Training: use 4 GPUs!
train_examples
40442
len(train_dataloader) 1264
args  {'workers': 2, 'num_train_epochs': 5, 'train_batch_size': 32, 'eval_batch_size': 8, 'weight_decay': 0, 'learning_rate': 4e-05, 'adam_epsilon': 1e-08, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'model_type': 'bert', 'model_name': 'bert-base-uncased', 'task_name': 'binary', 'output_mode': 'classification', 'max_seq_length': 512, 'fp16': False, 'fp16_opt_level': 'O1', 'gradient_accumulation_steps': 1, 'logging_steps': 500, 'save_steps': 1000, 'reprocess_input_data': False, 'hosts': ['algo-1'], 'current_host': 'algo-1', 'model_dir': '/opt/ml/model', 'data_dir': '/opt/ml/input/data/training', 'output_dir': './outputs', 'num_gpus': 4}
#0150.814048
algo-1:103:200 [0] misc/ibvwrap.cu:63 NCCL WARN Failed to open libibverbs.so[.1]
NCCL version 2.4.2+cuda9.0
#0150.477484#0150.367469#0150.269720#0150.171907#0150.178503#0150.170163#0150.117713#0150.060228#0150.057099#0150.045094#0150.142352#0150.137822#0150.245391#0150.068158

00430#0150.000361#0150.000589#0150.000541#0150.000490#0150.000709#0150.000560#0150.000550#0150.000624#0150.000772#0150.000618#0150.001025#0150.000651#0150.193930#0150.000654#0150.000654#0150.000688#0150.000877#0150.000672#0150.000926#0150.000945#0150.000817#0150.000677#0150.001088#0150.000844#0150.000726#0150.000647#0150.000664#0150.001161#0150.000698#0150.000710#0150.000698#0150.007471#0150.000499#0150.068379#0150.000802#0150.001051#0150.000693#0150.001734#0150.008825#0150.000578#0150.000639#0150.000471#0150.000432#0150.000700#0150.000788#0150.000770#0150.000509#0150.001070#0150.000672#0150.000558#0150.000571#0150.000331#0150.000374#0150.000351#0150.000337#0150.000338#0150.000365#0150.000589#0150.000321#0150.000426#0150.000791#0150.000320#0150.000636#0150.000444#0150.164614#0150.000304#0150.000316#0150.000302#0150.000306#0150.000299#0150.000300#0150.252525#0150.000636#0150.000408#0150.085941#0150.000518#0150.000580#0150.138061#0150.001156#0150.002467#0150.000752#0150.001753#0150.00087

8#0150.000112#0150.000101#0150.000094#0150.000439#0150.000105#0150.000101#0150.000624#0150.000178#0150.000265#0150.000097#0150.000092#0150.000091#0150.000095#0150.000094#0150.000094#0150.000091#0150.000631#0150.000095#0150.000100#0150.001553#0150.000190#0150.000095#0150.000312#0150.000090#0150.000095#0150.000099#0150.000094#0150.000091#0150.000508#0150.000090#0150.000086#0150.000088#0150.000087#0150.300914#0150.000575#0150.000112#0150.268246#0150.000157#0150.000197#0150.000729#0150.000770#0150.000295#0150.001117#0150.030973#0150.002573#0150.000381#0150.000854#0150.000407#0150.000425#0150.000436#0150.000479#0150.000666#0150.000406#0150.000410#0150.000403#0150.000596#0150.001021#0150.000594#0150.000572#0150.000547#0150.000369#0150.000301#0150.000609#0150.000568#0150.000512#0150.000308#0150.000241#0150.000247#0150.000250#0150.000243#0150.000731#0150.000253#0150.005904#0150.000447#0150.000238#0150.000251#0150.000239#0150.000960#0150.000232#0150.000479#0150.000410#0150.000407#0150.000749#01

In [ ]:
estimator.latest_training_job.name

# Import model into SageMaker
The PyTorch model uses a npy serializer and deserializer by default. since we have a custom implementation of all the hosting functions and plan on using JSON instead, we need a predictor that can serialize and deserialize JSON.

In [ ]:
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

class JSONPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

Since hosting functions implemented outside of train script we can't just use estimator object to deploy the model. Instead we need to create a PyTorchModel object using the latest training job to get the S3 location of the trained model data. Besides model data location in S3, we also need to configure PyTorchModel with the script and source directory (because our generate script requires model and data classes from source directory), an IAM role.

In [ ]:
from sagemaker.pytorch import PyTorchModel

#training_job_name = estimator.latest_training_job.name
training_job_name = "sagemaker-pytorch-2019-09-19-14-32-04-131"
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']
model = PyTorchModel(model_data=trained_model_location,
                     role=role,
                     framework_version='1.1.0',
                     entry_point='inference.py',
                     source_dir='email-compliance-bert',
                     #git_config=git_config,
                     predictor_cls=JSONPredictor)

Create endpoint
Now the model is ready to be deployed at a SageMaker endpoint and we are going to use the sagemaker.pytorch.model.PyTorchModel.deploy method to do this. We can use a CPU-based instance for inference (in this case an ml.m4.xlarge), even though we trained on GPU instances, because at the end of training we moved model to cpu before returning it. This way we can load trained model on any device and then move to GPU if CUDA is available.

In [ ]:
predictor = model.deploy(initial_instance_count=5, instance_type='ml.c5.2xlarge')
#predictor = model.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge')
#https://aws.amazon.com/machine-learning/elastic-inference/pricing/

In [ ]:
predictor.endpoint

In [ ]:
hello_song="""
Hello, it's me.
I was wondering if after all these years you'd like to meet.
To go over everything.
They say that time's supposed to heal you.
But I ain't done much healing.
"""

In [ ]:
import time
import json
start_t=time.time()
input_json = {
    'txt': hello_song
}
d=json.dumps(input_json)
#response = predictor.predict(input_json)
#print("response=", response, time.time()-start_t)

In [ ]:
import boto3
import json
runtime = boto3.Session().client(service_name='runtime.sagemaker',region_name='us-east-1')
#endpoint_name = 'sagemaker-pytorch-2019-09-15-13-33-39-536'
endpoint_name = 'SMEndpoint-Nwu6HNjfwGTn'
start_t=time.time()
response = runtime.invoke_endpoint(EndpointName=endpoint_name,
 ContentType='application/json',
 Body=d)
result = json.loads(response['Body'].read().decode())
print("response=", response, time.time()-start_t)

In [ ]:
print(result)

Cleanup
To delete the prediction endpoint to release the instance(s) associated with it.

In [ ]:
#sagemaker_session.delete_endpoint(predictor.endpoint)

In [ ]:
!more requirements.txt


In [ ]:
!cat Dockerfile

In [ ]:
 !./build_and_push.sh sagemaker-pytorch-email-compliance